<a href="https://colab.research.google.com/github/KadjelRamkisoen/BertIR/blob/main_kate/dev_set_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Path to data files
# IR_Path = '/content/drive/MyDrive/UvAntwerpen/2021-2022/Information Retrieval/Assignment 2/'


Mounted at /content/drive


In [ ]:
!nvidia-smi 

Wed Dec  1 13:25:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
IR_Path = '/content/drive/MyDrive/Information Retrieval/Assignment 2/'

In [ ]:
dev_data = 'dev_data.csv'
all_docs = 'all_docs.csv'
dev_queries = 'dev_queries.csv'
test_data = 'test_data.csv'
training_data = 'training_data.csv'
lucene_data = 'raw_dev_Lucene_retrievals.csv'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(IR_Path + dev_data,memory_map=True)
# df = pd.read_csv(IR_Path + all_docs)
# df = pd.read_csv(IR_Path + dev_queries)
# df = pd.read_csv(IR_Path + test_data)
# df = pd.read_csv(IR_Path + training_data)
# df = pd.read_csv(IR_Path + lucene_data)

In [ ]:
df.head()

,Query_number,doc_number,Query,doc_text,label
0,1089071,432658,va death benefits contact number,Veteran Benefits in Nebraska Ads You May Be In...,1
1,1089071,417115,va death benefits contact number,Home › Benefit Library › State / Territory Ben...,1
2,1089071,429474,va death benefits contact number,"VA Burial Benefits Burial Allowances Burial, H...",1
3,1089071,122086,va death benefits contact number,Home › Benefit Library › Federal Benefits › Bu...,1
4,1089071,60461,va death benefits contact number,Call Today1-844-VET-LAWS (838-5297)REFER A FRI...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19992 entries, 0 to 19991
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Query_number  19992 non-null  int64 
 1   doc_number    19992 non-null  int64 
 2   Query         19992 non-null  object
 3   doc_text      19992 non-null  object
 4   label         19992 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 781.1+ KB


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 3.1 MB 16.5 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 1.2 MB 38.5 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=20253a91dca2cd9e939ce5807dbf2e182e429eae70941873dc4d3dbaefd889b8
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install faiss-gpu

     |████████████████████████████████| 90.1 MB 21 kB/s 


In [ ]:

import time
from tqdm import tqdm
# from textblob import TextBlob
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/554 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
data = df

In [ ]:
import gc
df = data[['doc_number','doc_text']]

In [ ]:
df.shape

(19992, 2)

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['doc_text'],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.shape

(18260, 2)

In [ ]:
import faiss

In [ ]:
encoded_data = model.encode(df["doc_text"].tolist())

In [ ]:
encoded_data = np.asarray(encoded_data.astype('float32'))

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [ ]:
index.add_with_ids(encoded_data, np.array(range(0, len(df))))

In [ ]:
# faiss.write_index(index, 'movie_plot.index')

## !! Deserializing the index
# index = faiss.read_index('abc_news')
faiss.write_index(index, IR_Path + 'doc_text.index')

In [ ]:
from pprint import pprint

query="usf admissions office phone number"
results=search(query, top_k=20, index=index, model=model)

print("\n")
for result in results:
    print('\t',pprint(result))

>>>> Results in Total Time: 0.03328895568847656


{'doc_number': 408240,
 'doc_text': 'Visit Admissions Office Usf on the Given Address:4202 E. Fowler '
             'Avenue, SVC 1036 Tampa, FL 33620-6900Contact Admissions Office '
             'Usf on the Given Contact Number:813-974-3350. If the contact '
             'number or email address of Admissions Office Usf is incorrect, '
             'please tell us HERESend Admissions Office Usf Email on given '
             'Email Address:admissions@usf.edu Open Admissions Office Usf '
             'Website by given Website Address:www.usf.edu Was your query '
             'resolved after making call or sending email to Admissions Office '
             'Usf'}
	 None
{'doc_number': 131285,
 'doc_text': 'Admissions Freshman Freshman Admissions Majors & Minors Freshman '
             'Orientation Summer Beginnings Dual Enrollment at '
             'USFSMInternational Students Transfer Transfer Admissions '
             'Programs & Majors T

In [ ]:
def fetch_doc_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['doc_number'] = info['doc_number']
    meta_dict['doc_text'] = info['doc_text'][:500]
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_doc_info(idx) for idx in top_k_ids]
    return results

In [ ]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6', max_length=512)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/541 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

model_inputs = [[query,item['doc_text']] for item in results]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'doc_number': inp['doc_number'], 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)


In [ ]:
print("\n")
for result in ranked_results:
    print('\t',pprint(result))



{'Score': 0.9886374, 'doc_number': 408240}
	 None
{'Score': 0.048137262, 'doc_number': 295065}
	 None
{'Score': 0.00066198315, 'doc_number': 409156}
	 None
{'Score': 0.0006192998, 'doc_number': 48247}
	 None
{'Score': 0.00041761904, 'doc_number': 53312}
	 None
{'Score': 0.00032994314, 'doc_number': 77126}
	 None
{'Score': 0.00026736272, 'doc_number': 482858}
	 None
{'Score': 0.00024996762, 'doc_number': 131285}
	 None
{'Score': 0.00023856515, 'doc_number': 320477}
	 None
{'Score': 0.00021945377, 'doc_number': 305125}
	 None
{'Score': 0.0002098585, 'doc_number': 402173}
	 None
{'Score': 0.00018473559, 'doc_number': 444991}
	 None
{'Score': 0.00018146947, 'doc_number': 32247}
	 None
{'Score': 0.0001779673, 'doc_number': 454583}
	 None
{'Score': 0.00017682875, 'doc_number': 310143}
	 None
{'Score': 0.00017135573, 'doc_number': 267221}
	 None
{'Score': 0.00016942497, 'doc_number': 98231}
	 None
{'Score': 0.0001623641, 'doc_number': 407263}
	 None
{'Score': 0.0001606999, 'doc_number': 397

In [ ]:
!pip install bert-score

     |████████████████████████████████| 59 kB 3.1 MB/s 


In [ ]:
from bert_score import score

In [ ]:
ref=["usf admissions office phone number"]

In [ ]:
ranked_results_bert = []

for cand in results:
    P, R, F1 = score([cand['doc_text']], ref, lang='en')
    ranked_results_bert.append({'doc_number': cand['doc_number'], 'Score': F1.numpy()[0]})
    

In [ ]:
ranked_results_bert

[{'Score': 0.80063784, 'doc_number': 408240},
 {'Score': 0.80008984, 'doc_number': 407263},
 {'Score': 0.7980095, 'doc_number': 305125},
 {'Score': 0.7952826, 'doc_number': 77126},
 {'Score': 0.7950419, 'doc_number': 295065},
 {'Score': 0.79476136, 'doc_number': 320477},
 {'Score': 0.79434323, 'doc_number': 454583},
 {'Score': 0.79365927, 'doc_number': 310143},
 {'Score': 0.7934236, 'doc_number': 409156},
 {'Score': 0.7928563, 'doc_number': 397059},
 {'Score': 0.79034084, 'doc_number': 53312},
 {'Score': 0.7875394, 'doc_number': 48247},
 {'Score': 0.7858844, 'doc_number': 267221},
 {'Score': 0.7825909, 'doc_number': 131285},
 {'Score': 0.7799705, 'doc_number': 32247},
 {'Score': 0.77806216, 'doc_number': 444991},
 {'Score': 0.77759796, 'doc_number': 412272},
 {'Score': 0.7750369, 'doc_number': 402173},
 {'Score': 0.7733984, 'doc_number': 482858},
 {'Score': 0.7454754, 'doc_number': 98231}]

In [ ]:
#Sort the scores in decreasing order
ranked_results_bert = sorted(ranked_results_bert, key=lambda x: x['Score'], reverse=True)
print("\n")
for result in ranked_results_bert:
    print('\t',pprint(result))



{'Score': 0.80063784, 'doc_number': 408240}
	 None
{'Score': 0.80008984, 'doc_number': 407263}
	 None
{'Score': 0.7980095, 'doc_number': 305125}
	 None
{'Score': 0.7952826, 'doc_number': 77126}
	 None
{'Score': 0.7950419, 'doc_number': 295065}
	 None
{'Score': 0.79476136, 'doc_number': 320477}
	 None
{'Score': 0.79434323, 'doc_number': 454583}
	 None
{'Score': 0.79365927, 'doc_number': 310143}
	 None
{'Score': 0.7934236, 'doc_number': 409156}
	 None
{'Score': 0.7928563, 'doc_number': 397059}
	 None
{'Score': 0.79034084, 'doc_number': 53312}
	 None
{'Score': 0.7875394, 'doc_number': 48247}
	 None
{'Score': 0.7858844, 'doc_number': 267221}
	 None
{'Score': 0.7825909, 'doc_number': 131285}
	 None
{'Score': 0.7799705, 'doc_number': 32247}
	 None
{'Score': 0.77806216, 'doc_number': 444991}
	 None
{'Score': 0.77759796, 'doc_number': 412272}
	 None
{'Score': 0.7750369, 'doc_number': 402173}
	 None
{'Score': 0.7733984, 'doc_number': 482858}
	 None
{'Score': 0.7454754, 'doc_number': 98231}
	 

In [ ]:
final_results = pd.DataFrame()
final_results['faiss_ranking'] = [item['doc_number'] for item in results]
final_results['cross_encoder'] = [item['doc_number'] for item in ranked_results]
final_results['bert_score'] = [item['doc_number'] for item in ranked_results_bert]


In [ ]:
final_results

,faiss_ranking,cross_encoder,bert_score
0,408240,408240,408240
1,131285,295065,407263
2,310143,409156,305125
3,402173,48247,77126
4,295065,53312,295065
5,267221,77126,320477
6,77126,482858,454583
7,98231,131285,310143
8,454583,320477,409156
9,48247,305125,397059


In [ ]:
data.head()

,Query_number,doc_number,Query,doc_text,label,rank
0,1089071,432658,va death benefits contact number,Veteran Benefits in Nebraska Ads You May Be In...,1,1
1,1089071,417115,va death benefits contact number,Home › Benefit Library › State / Territory Ben...,1,2
2,1089071,429474,va death benefits contact number,"VA Burial Benefits Burial Allowances Burial, H...",1,3
3,1089071,122086,va death benefits contact number,Home › Benefit Library › Federal Benefits › Bu...,1,4
4,1089071,60461,va death benefits contact number,Call Today1-844-VET-LAWS (838-5297)REFER A FRI...,1,5


In [ ]:
data['rank'] = data.groupby(['Query_number']).cumcount()+1 

In [ ]:
one_q=data[data["Query_number"] == 1089085]

In [ ]:
one_q["faiss_ranking"] = final_results["faiss_ranking"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# top 14 are good

In [ ]:
final_results.head()

,faiss_ranking,cross_encoder,bert_score
0,408240,408240,408240
1,131285,295065,407263
2,310143,409156,305125
3,402173,48247,77126
4,295065,53312,295065


In [ ]:
merged = pd.merge(final_results, one_q[['doc_number', 'rank']],  how='left', left_on=['faiss_ranking'], right_on = ['doc_number'], suffixes=('', 'faiss'))
merged = pd.merge(merged, one_q[['doc_number', 'rank']],  how='left', left_on=['cross_encoder'], right_on = ['doc_number'], suffixes=('', 'cross'))
merged = pd.merge(merged, one_q[['doc_number', 'rank']],  how='left', left_on=['bert_score'], right_on = ['doc_number'], suffixes=('', 'bert'))

KeyError: ignored

In [ ]:
merged

,faiss_ranking,cross_encoder,bert_score,doc_number,rank,doc_numbercross,rankcross,doc_numberbert,rankbert
0,408240,408240,408240,408240.0,1.0,408240.0,1.0,408240.0,1.0
1,131285,295065,407263,131285.0,4.0,295065.0,30.0,NaN,NaN
2,310143,409156,305125,310143.0,8.0,NaN,NaN,NaN,NaN
3,402173,48247,77126,402173.0,22.0,NaN,NaN,NaN,NaN
4,295065,53312,295065,295065.0,30.0,NaN,NaN,295065.0,30.0
5,267221,77126,320477,NaN,NaN,NaN,NaN,NaN,NaN
6,77126,482858,454583,NaN,NaN,NaN,NaN,NaN,NaN
7,98231,131285,310143,NaN,NaN,131285.0,4.0,310143.0,8.0
8,454583,320477,409156,NaN,NaN,NaN,NaN,NaN,NaN
9,48247,305125,397059,NaN,NaN,NaN,NaN,NaN,NaN
